## Database Engineer Capstone Week 3

***

### Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import scipy as sp


import datetime
from datetime import datetime, timedelta
from scipy import stats

import mysql.connector as connector

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
#sets the default autosave frequency in seconds
%autosave 60 
sns.set_style('dark')
sns.set(font_scale=1.2)

plt.rc('axes', titlesize=9)
plt.rc('axes', labelsize=14)
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)

pd.set_option('display.max_columns',None)
#pd.set_option('display.max_rows',None)
pd.set_option('display.width', 1000)
pd.option_context('float_format','{:.2f}'.format)

random.seed(0)
np.random.seed(0)
np.set_printoptions(suppress=True)

Autosaving every 60 seconds


## Load Data from SQL database

Your first step is to import the connector module, enter your user details and connect with the database (Hint: you can use an alias when importing the module).

### MySQL

In [2]:
#Create a connection

mydb = connector.connect(
  user="sqluser",
  passwd="password",
  database="littlelemondb",
  auth_plugin='mysql_native_password'  
)

In [3]:
print(mydb)

In [4]:
mycursor = mydb.cursor()

In [5]:
mycursor

In [6]:
mydb.reconnect() #Reconnect cursor

In this second task, you now need to query the database to show all tables within the database. 

In [7]:
pd.read_sql_query('SHOW tables', mydb)

,Tables_in_littlelemondb
0,booking
1,customer
2,deliverystatus
3,menu
4,orders
5,ordersview
6,ordersview2
7,staff


In [8]:
pd.read_sql_query("SHOW columns FROM booking", mydb)

,Field,Type,Null,Key,Default,Extra
0,BookingID,int,NO,PRI,None,
1,BookingDate,datetime,NO,,None,
2,TableNumber,int,NO,,None,
3,CustomerID,int,YES,,None,
4,StaffID,int,YES,,None,


An alternate way to learn the same information would be to use the DESCRIBE function. The syntax is:

In [9]:
pd.read_sql_query("DESCRIBE booking", mydb)

,Field,Type,Null,Key,Default,Extra
0,BookingID,int,NO,PRI,None,
1,BookingDate,datetime,NO,,None,
2,TableNumber,int,NO,,None,
3,CustomerID,int,YES,,None,
4,StaffID,int,YES,,None,


For the third and final task, Little Lemon need you to return specific details from your database. They require the full name and contact details for every customer that has placed an order greater than $60 for a promotional campaign. 

In [10]:
pd.read_sql_query("SELECT * FROM customer", mydb)

,CustomerID,FullName,ContactNumber,Email
0,1,John Doe,12627682932,john.doe@example.com
1,2,Jane Doe,12395875561,jane.doe@example.com
2,3,Alice,16262786391,alice@example.com
3,4,Bob,17736726363,bob@example.com
4,5,Charlie,16054610849,charlie@example.com
5,6,David,17635930461,david@example.com
6,7,Emily,12519265580,emily@example.com
7,8,Frank,16187625501,frank@example.com
8,9,Grace,14098874947,grace@example.com
9,10,Hannah,12402270015,hannah@example.com


In [11]:
pd.read_sql_query("SELECT * FROM orders", mydb)

,OrderID,OrderDate,Quantity,TotalCost,Booking_BookingID,Customer_CustomerID,Menu_MenuID,DeliveryStatus_DeliveryID
0,1,2023-09-01 12:00:00,3,499.0,None,1,5,2
1,2,2023-09-01 12:30:00,2,295.0,None,2,8,3
2,3,2023-09-02 13:00:00,4,599.0,None,3,4,4
3,4,2023-09-02 14:00:00,1,199.0,None,4,7,7
4,5,2023-09-03 15:00:00,5,795.0,None,5,1,87
5,6,2023-09-03 16:00:00,2,295.0,None,6,9,34
6,7,2023-09-04 17:00:00,3,499.0,None,7,2,256
7,8,2023-09-04 18:00:00,4,599.0,None,8,6,35
8,9,2023-09-05 20:00:00,1,199.0,None,9,3,34
9,10,2023-09-05 20:00:00,5,795.0,None,10,10,32


In [12]:
pd.read_sql_query("""SELECT customer.FullName, customer.ContactNumber FROM customer
                    INNER JOIN orders
                    ON customer.CustomerID = orders.Customer_CustomerID
                    WHERE orders.TotalCost > 60
                    """, mydb)

,FullName,ContactNumber
0,John Doe,12627682932
1,Jane Doe,12395875561
2,Alice,16262786391
3,Bob,17736726363
4,Charlie,16054610849
5,David,17635930461
6,Emily,12519265580
7,Frank,16187625501
8,Grace,14098874947
9,Hannah,12402270015


The cloned repository contains a procedure called GetMaxQuantity(). Call this procedure and verify 

In [13]:
mydb.reconnect() #Reconnect cursor
pd.read_sql_query("""CALL GetMaxQuantity();""", mydb)

,MAX(orders.Quantity)
0,5


Call the ManageBooking() procedure by passing the appropriate parameters. First with an available table number, then with one that has already been reserved.

In [14]:
mydb.reconnect() #Reconnect cursor
pd.read_sql_query("""CALL CheckBooking("2022-12-17", 5);""", mydb)

,Table Status
0,Table is already booked.


Create an SQL statement that calls the AddBooking() procedure

Call the cursor .execute() method using the above parameters. Print out the result using the cursor .fetchall() method.

In [15]:
mydb.reconnect() #Reconnect cursor
pd.read_sql_query("""CALL AddBooking(99, 99, "2024-12-10",15,26);""", mydb)


,Confirmation
0,New booking added


Create a SQL statement that calls the UpdateBooking() procedure

In [16]:
mydb.reconnect() #Reconnect cursor
pd.read_sql_query("""CALL UpdateBooking(99, "2024-01-10");""", mydb)

,Confirmation
0,Booking99updated


In [17]:
mydb.reconnect() #Reconnect cursor
pd.read_sql_query("""SELECT * FROM Booking;""", mydb)

,BookingID,BookingDate,TableNumber,CustomerID,StaffID
0,1,2022-12-17,5,1.0,0.0
1,2,2022-12-17,3,3.0,0.0
2,3,2022-12-17,2,2.0,0.0
3,4,2022-12-17,2,1.0,0.0
4,6,2022-12-17,20,NaN,NaN
5,7,2022-12-17,22,NaN,NaN
6,8,2022-12-17,24,NaN,NaN
7,10,2022-12-17,28,NaN,NaN
8,11,2022-12-17,10,NaN,NaN
9,12,2022-12-17,12,NaN,NaN


Create a SQL statement that calls the CancelBooking() procedure

In [18]:
mydb.reconnect() #Reconnect cursor
pd.read_sql_query("""CALL CancelBooking(99);""", mydb)

,Confirmation
0,Booking 99 cancelled


In [19]:
mydb.reconnect() #Reconnect cursor
pd.read_sql_query("""SELECT * FROM Booking;""", mydb)

,BookingID,BookingDate,TableNumber,CustomerID,StaffID
0,1,2022-12-17,5,1.0,0.0
1,2,2022-12-17,3,3.0,0.0
2,3,2022-12-17,2,2.0,0.0
3,4,2022-12-17,2,1.0,0.0
4,6,2022-12-17,20,NaN,NaN
5,7,2022-12-17,22,NaN,NaN
6,8,2022-12-17,24,NaN,NaN
7,10,2022-12-17,28,NaN,NaN
8,11,2022-12-17,10,NaN,NaN
9,12,2022-12-17,12,NaN,NaN


In [20]:
mydb.close()